In [1]:
import threading
import uuid

In [2]:
"""Limit and Capacities"""

warehouse_capacity = 100_000_000
warehouse_quantity = 80_000_000
successful_transactions = 0
failed_transactions = 0

In [3]:
""" defining the rack """

"""
1) putting id using uuid 
2) capacity and volume 
3) rack type (in or out)
4) mutex or a thread  lock
"""
class Rack:
    def __init__(self, capacity, quantity, rack_type):
        self.id = uuid.uuid4()
        self.capacity = int(capacity)
        self.quantity = int(quantity)
        self.rack_type = rack_type
        self.lock = threading.Lock()
        self.is_locked = False

        if self.quantity > self.capacity:
            raise ValueError(f"Invalid Rack: {rack_type} rack has quantity {self.quantity} exceeding capacity {self.capacity}")

        if self.quantity < 0:
            raise ValueError(f"Invalid Rack: {rack_type} rack has negative quantity {self.quantity}")

    def is_full(self):
        return self.quantity >= self.capacity

    def remaining_capacity(self):
        return self.capacity - self.quantity

    def acquire_lock(self, timeout=1):
        if self.lock.acquire(timeout=timeout):
            self.is_locked = True
            return True
        return False

    def release_lock(self):
        if self.is_locked:
            self.lock.release()
            self.is_locked = False

    def __repr__(self):
        return (f"Rack(ID: {self.id}, Type: {self.rack_type}, "
                f"Capacity: {self.capacity / 1_000_000}g, Quantity: {self.quantity / 1_000_000}g, "
                f"Remaining: {self.remaining_capacity() / 1_000_000}g, Full: {self.is_full()}, Locked: {self.is_locked})")


In [4]:
warehouse = Rack(warehouse_capacity, warehouse_quantity, "WAREHOUSE")
warehouse_lock = threading.Lock()

In [5]:
out_racks = {
    Rack(10_000_000, 5_000_000, "OUT"),
    Rack(15_000_000, 8_000_000, "OUT"),
    Rack(20_000_000, 18_000_000, "OUT"),
    Rack(25_000_000, 10_000_000, "OUT"),
    Rack(30_000_000, 28_000_000, "OUT")
}

in_racks = {
    Rack(20_000_000, 10_000_000, "IN"),
    Rack(25_000_000, 20_000_000, "IN"),
    Rack(30_000_000, 15_000_000, "IN"),
    Rack(35_000_000, 30_000_000, "IN"),
    Rack(40_000_000, 18_000_000, "IN")
}

In [6]:
""" defining the set beacasue of 0(1) seach complexity"""
locked_racks = set()

unlocked_racks = set()

for rack in in_racks:
    unlocked_racks.add(rack)
for rack in out_racks:
    unlocked_racks.add(rack)

In [7]:
"""All counter global variables"""


successful_rebalance_count = 0  
failed_rebalance_count = 0

In [8]:
def rebalance_out_for_gold(rack, required_gold, warehouse):
    global successful_rebalance_count, failed_rebalance_count

    gold_needed = required_gold - rack.quantity

    # Lock warehouse first
    if not warehouse_lock.acquire(timeout=1):
        failed_rebalance_count += 1
        return False

    # Lock rack second
    if not rack.lock.acquire(timeout=1):
        warehouse_lock.release()
        failed_rebalance_count += 1
        return False

    try:
        if warehouse.quantity < gold_needed or (rack.quantity + gold_needed) > rack.capacity:
            failed_rebalance_count += 1
            # If rebalance fails, unlock rack
            rack.lock.release()
            warehouse_lock.release()
            locked_racks.discard(rack)
            unlocked_racks.add(rack)
            return False

        warehouse.quantity -= gold_needed
        rack.quantity += gold_needed
        successful_rebalance_count += 1

        # DO NOT UNLOCK rack if success
        # Only unlock warehouse here
        warehouse_lock.release()
        return True
    except:
        # On exception, release both
        rack.lock.release()
        warehouse_lock.release()
        locked_racks.discard(rack)
        unlocked_racks.add(rack)
        raise

In [9]:

def rebalance_out_for_capacity(rack, required_required, warehouse):
    global successful_rebalance_count, failed_rebalance_count

    space_needed = required_required - rack.remaining_capacity()

    # Lock warehouse first
    if not warehouse_lock.acquire(timeout=1):
        failed_rebalance_count += 1
        return False

    # Lock rack second
    if not rack.lock.acquire(timeout=1):
        warehouse_lock.release()
        failed_rebalance_count += 1
        return False

    try:
        if warehouse.remaining_capacity() < space_needed or rack.quantity < space_needed:
            failed_rebalance_count += 1
            # If rebalance fails, unlock rack
            rack.lock.release()
            warehouse_lock.release()
            locked_racks.discard(rack)
            unlocked_racks.add(rack)
            return False

        warehouse.quantity += space_needed
        rack.quantity -= space_needed
        successful_rebalance_count += 1

        # DO NOT UNLOCK rack if success
        # Only unlock warehouse here
        warehouse_lock.release()
        return True
    except:
        # On exception, release both
        rack.lock.release()
        warehouse_lock.release()
        locked_racks.discard(rack)
        unlocked_racks.add(rack)
        raise

In [10]:
def select_source_rack(racks, required_gold, rack_type, warehouse):
    if rack_type == "OUT":  # Case A: BUY Transaction (Source is OUT)
        candidate_racks = []
        for rack in unlocked_racks:
            if rack.rack_type == "OUT" and rack.quantity >= required_gold:
                candidate_racks.append(rack)

        if candidate_racks:
            best_rack = candidate_racks[0]
            for rack in candidate_racks:
                if rack.remaining_capacity() < best_rack.remaining_capacity():
                    best_rack = rack

            if best_rack.lock.acquire():
                unlocked_racks.remove(best_rack)
                locked_racks.add(best_rack)
                return best_rack

        # No suitable OUT rack found, attempt rebalancing
        best_rebalance_candidate = None
        max_quantity = -1

        for rack in unlocked_racks:
            if rack.rack_type == "OUT" and rack.quantity < required_gold:
                if rack.quantity > max_quantity:
                    best_rebalance_candidate = rack
                    max_quantity = rack.quantity

        if best_rebalance_candidate and best_rebalance_candidate.lock.acquire():
            unlocked_racks.remove(best_rebalance_candidate)
            locked_racks.add(best_rebalance_candidate)
            success = rebalance_out_for_gold(best_rebalance_candidate, required_gold, warehouse)
            if success:
                return best_rebalance_candidate

            best_rebalance_candidate.lock.release()
            locked_racks.remove(best_rebalance_candidate)
            unlocked_racks.add(best_rebalance_candidate)

        return None  # No valid OUT rack found

    else:  # Case B: SELL Transaction (Source is IN)
        candidate_racks = []
        for rack in unlocked_racks:
            if rack.rack_type == "IN" and rack.quantity >= required_gold:
                candidate_racks.append(rack)

        if candidate_racks:
            best_rack = candidate_racks[0]
            for rack in candidate_racks:
                if rack.remaining_capacity() < best_rack.remaining_capacity():
                    best_rack = rack

            if best_rack.lock.acquire():
                unlocked_racks.remove(best_rack)
                locked_racks.add(best_rack)
                return best_rack

        return None  # No valid IN rack found, and IN racks cannot rebalance

In [11]:
def select_destination_rack(required_gold, rack_type, warehouse):
    if rack_type == "OUT":  # Case A: SELL Transaction (Destination is OUT)
        candidate_racks = []
        for rack in unlocked_racks:
            if rack.rack_type == "OUT" and rack.remaining_capacity() >= required_gold:
                candidate_racks.append(rack)

        if candidate_racks:
            best_rack = candidate_racks[0]
            for rack in candidate_racks:
                if rack.remaining_capacity() > best_rack.remaining_capacity():
                    best_rack = rack

            if best_rack.lock.acquire():
                unlocked_racks.remove(best_rack)
                locked_racks.add(best_rack)
                return best_rack

        # No suitable OUT rack found, attempt rebalancing
        best_rebalance_candidate = None
        highest_capacity = -1

        for rack in unlocked_racks:
            if rack.rack_type == "OUT" and rack.remaining_capacity() < required_gold:
                if rack.capacity > highest_capacity:
                    best_rebalance_candidate = rack
                    highest_capacity = rack.capacity

        if best_rebalance_candidate and best_rebalance_candidate.lock.acquire():
            unlocked_racks.remove(best_rebalance_candidate)
            locked_racks.add(best_rebalance_candidate)
            success = rebalance_out_for_capacity(best_rebalance_candidate, required_gold, warehouse)
            if success:
                return best_rebalance_candidate

            best_rebalance_candidate.lock.release()
            locked_racks.remove(best_rebalance_candidate)
            unlocked_racks.add(best_rebalance_candidate)

        return None  # No valid OUT rack found

    else:  # Case B: BUY Transaction (Destination is IN)
        candidate_racks = []
        for rack in unlocked_racks:
            if rack.rack_type == "IN" and rack.remaining_capacity() >= required_gold:
                candidate_racks.append(rack)

        if candidate_racks:
            best_rack = candidate_racks[0]
            for rack in candidate_racks:
                if rack.remaining_capacity() > best_rack.remaining_capacity():
                    best_rack = rack

            if best_rack.lock.acquire():
                unlocked_racks.remove(best_rack)
                locked_racks.add(best_rack)
                return best_rack

        return None  # No valid IN rack found, and IN racks cannot rebalance

In [12]:
def buy_gold(amount, out_racks, in_racks, warehouse):
    global successful_transactions, failed_transactions
    source = select_source_rack(out_racks, amount, "OUT", warehouse)
    if not source:
        failed_transactions += 1
        return False, None, None
    destination = select_destination_rack(amount, "IN", warehouse)
    if not destination:
        failed_transactions += 1
        source.release_lock()
        locked_racks.discard(source)
        unlocked_racks.add(source)
        return False, None, None
    a, b = (source, destination) if id(source) < id(destination) else (destination, source)
    if not a.acquire_lock(timeout=1):
        failed_transactions += 1
        return False, None, None
    if not b.acquire_lock(timeout=1):
        a.release_lock()
        failed_transactions += 1
        return False, None, None
    try:
        if source.quantity < amount or destination.quantity + amount > destination.capacity:
            failed_transactions += 1
            return False, source, destination
        source.quantity -= amount
        destination.quantity += amount
        successful_transactions += 1
        return True, source, destination
    finally:
        b.release_lock()
        a.release_lock()
        locked_racks.discard(source)
        locked_racks.discard(destination)
        unlocked_racks.add(source)
        unlocked_racks.add(destination)

In [13]:
def sell_gold(amount, in_racks, out_racks, warehouse):
    global successful_transactions, failed_transactions
    source = select_source_rack(in_racks, amount, "IN", warehouse)
    if not source:
        failed_transactions += 1
        return False, None, None
    destination = select_destination_rack(amount, "OUT", warehouse)
    if not destination:
        failed_transactions += 1
        source.release_lock()
        locked_racks.discard(source)
        unlocked_racks.add(source)
        return False, None, None
    a, b = (source, destination) if id(source) < id(destination) else (destination, source)
    if not a.acquire_lock(timeout=1):
        failed_transactions += 1
        return False, None, None
    if not b.acquire_lock(timeout=1):
        a.release_lock()
        failed_transactions += 1
        return False, None, None
    try:
        if source.quantity < amount or destination.quantity + amount > destination.capacity:
            failed_transactions += 1
            return False, source, destination
        source.quantity -= amount
        destination.quantity += amount
        successful_transactions += 1
        return True, source, destination
    finally:
        b.release_lock()
        a.release_lock()
        locked_racks.discard(source)
        locked_racks.discard(destination)
        unlocked_racks.add(source)
        unlocked_racks.add(destination)

In [14]:
import random

def generate_transaction(ratio_buy, ratio_sell, total_transactions, buy_min, buy_max, sell_min, sell_max):
    transactions = []

    total_ratio = ratio_buy + ratio_sell
    if total_ratio == 0 or total_transactions <= 0:
        print("WARNING: No transactions generated due to zero ratio or total.")
        return transactions

    buy_count = round((ratio_buy / total_ratio) * total_transactions)
    sell_count = total_transactions - buy_count

    for _ in range(buy_count):
        amount_g = round(random.uniform(buy_min, buy_max), 4)
        amount_µg = int(amount_g * 1_000_000)
        transactions.append(("BUY", amount_µg))

    for _ in range(sell_count):
        amount_g = round(random.uniform(sell_min, sell_max), 4)
        amount_µg = int(amount_g * 1_000_000)
        transactions.append(("SELL", amount_µg))

    random.shuffle(transactions)
    return transactions

In [15]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def run_simulation(ratio_buy, ratio_sell, total_transactions, buy_min, buy_max, sell_min, sell_max):
    global successful_transactions, failed_transactions

    print("\n=== INITIAL STATE ===")
    print("\nIN Racks:", *in_racks, sep="\n")
    print("\nOUT Racks:", *out_racks, sep="\n")
    print(f"\nWarehouse: {warehouse}")

    transactions = generate_transaction(ratio_buy, ratio_sell, total_transactions, buy_min, buy_max, sell_min, sell_max)

    if not transactions:
        print("\nWARNING: No valid transactions generated. Exiting simulation.")
        return

    with ThreadPoolExecutor(max_workers=5) as executor:
        future_map = {}
        for tx_type, amount_µg in transactions:
            amount_g = amount_µg / 1_000_000
            print(f"\nGenerated Transaction: {tx_type} {amount_g:.6f}g ({amount_µg} µg)")

            if tx_type == "BUY":
                f = executor.submit(buy_gold, amount_µg, out_racks, in_racks, warehouse)
            else:
                f = executor.submit(sell_gold, amount_µg, in_racks, out_racks, warehouse)

            future_map[f] = (tx_type, amount_g, amount_µg)

        for future in as_completed(future_map):
            tx_type, amount_g, amount_µg = future_map[future]
            try:
                success, source, destination = future.result()
                if success:
                    print(f"Transaction {tx_type} {amount_g:.6f}g SUCCESS")
                else:
                    print(f"Transaction {tx_type} {amount_g:.6f}g FAILED")
            except Exception as e:
                print(f"Transaction {tx_type} {amount_g:.6f}g EXCEPTION: {e}")

    print("\n=== FINAL STATE ===")
    print("\nIN Racks:", *in_racks, sep="\n")
    print("\nOUT Racks:", *out_racks, sep="\n")
    print(f"\nWarehouse: {warehouse}")

    print(f"\nTotal Successful Transactions: {successful_transactions}")
    print(f"Total Failed Transactions: {failed_transactions}")
    print(f"Total Successful Rebalances: {successful_rebalance_count}")
    print(f"Total Failed Rebalances: {failed_rebalance_count}")

In [16]:
run_simulation(
    ratio_buy=2,
    ratio_sell=1,
    total_transactions=10,
    buy_min=0.0001,   # 0.0001g (100 µg)
    buy_max=40.0,     # 50g 
    sell_min=0.001,   # 1000 µg
    sell_max=40.0
)


=== INITIAL STATE ===

IN Racks:
Rack(ID: 94260b3d-7c44-49f0-ad5f-ad97c75352d9, Type: IN, Capacity: 20.0g, Quantity: 10.0g, Remaining: 10.0g, Full: False, Locked: False)
Rack(ID: 66208f2c-e17a-4c28-b73a-38546b70fb54, Type: IN, Capacity: 35.0g, Quantity: 30.0g, Remaining: 5.0g, Full: False, Locked: False)
Rack(ID: 3eb349e1-9745-4755-ba18-8b108a518cb4, Type: IN, Capacity: 25.0g, Quantity: 20.0g, Remaining: 5.0g, Full: False, Locked: False)
Rack(ID: e26df33c-eb9a-4033-a878-0e4f233dd62c, Type: IN, Capacity: 40.0g, Quantity: 18.0g, Remaining: 22.0g, Full: False, Locked: False)
Rack(ID: cfb9efa7-86d1-4f0a-9044-e62b1ed759d4, Type: IN, Capacity: 30.0g, Quantity: 15.0g, Remaining: 15.0g, Full: False, Locked: False)

OUT Racks:
Rack(ID: 457fedc6-5f33-4b38-a428-60bad61a33b6, Type: OUT, Capacity: 30.0g, Quantity: 28.0g, Remaining: 2.0g, Full: False, Locked: False)
Rack(ID: 26435139-ed7f-441f-80ba-9852ffff66a9, Type: OUT, Capacity: 10.0g, Quantity: 5.0g, Remaining: 5.0g, Full: False, Locked: False